## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

In [25]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

## Данные

In [66]:
stops = set(stopwords.words('russian'))
stops |= set(stopwords.words('english'))

In [15]:
#stops = set(stopwords.words('russian')) | {'gt',}

def remove_tags(text):
    text = re.sub('&gt;', '', text)
    text = re.sub('&lt;', '', text)
    text = re.sub('\\t', '', text)
    return re.sub(r'<[^>]+>', '', text)

def normalize(words):
    norm_words = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return norm_words

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [145]:
morph.parse('jc')[0].tag

OpencorporaTag('LATN')

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); -- + 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); --+

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); -- + 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [6]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [tokenize(remove_tags(text.lower())) for text in texts]

CPU times: user 3.23 s, sys: 465 ms, total: 3.69 s
Wall time: 3.78 s


In [ ]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [normalize(tokenize(text.lower())) for text in texts]

In [28]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

CPU times: user 13.1 s, sys: 747 ms, total: 13.8 s
Wall time: 14.1 s


In [30]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

#### 3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова);

In [67]:
all_t = Counter([])
for lst in texts:
    all_t += Counter(lst)

In [68]:
all_n = Counter([])
for lst in ngrammed_texts:
    all_n += Counter(lst)

In [71]:
stops_t = {i[0] for i in all_t.most_common(50)}
stops_n = {i[0] for i in all_n.most_common(50)}
least_n = {i[0] for i in all_n.most_common()[:-100-1:-1]}
least_t = {i[0] for i in all_n.most_common()[:-100-1:-1]}

In [72]:
stops = stops | new_stops_t | new_stops_n | least_n | least_t

In [73]:
len(stops)

534

In [74]:
%%time
texts_new_stops = open('habr_texts.txt').read().splitlines()
texts_new_stops = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts_new_stops], 30000)
ph = gensim.models.Phrases(texts_new_stops, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts_new_stops = p[texts_new_stops]

CPU times: user 44.2 s, sys: 1.28 s, total: 45.5 s
Wall time: 49 s


In [79]:
texts = ngrammed_texts_new_stops

In [80]:
del texts_new_stops

## 4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем;

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [89]:
dictinary = gensim.corpora.Dictionary(texts)

In [90]:
dictinary.filter_extremes(no_above=0.3)
dictinary.compactify()

In [91]:
print(dictinary)

Dictionary(15489 unique tokens: ['2-х', '3', 'address', 'api', 'assembly']...)


Преобразуем наши тексты в мешки слов. 

In [92]:
corpus = [dictinary.doc2bow(text) for text in texts]
# если текстов много, то тут может быть генератор

In [ ]:
?gensim.models.LdaMulticore

In [ ]:
%%time
lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, passes=5, workers=2, eta='auto', iterations=10) # если поддерживается многопоточность 
#lda = gensim.models.LdaModel(200, id2word=dictinary, passes=5)

In [161]:
def my_lda(texts, num_topics=10, passes=10, eta='auto', tf=False):
            
    dictinary = gensim.corpora.Dictionary(texts)
    dictinary.filter_extremes(no_above=0.3)
    dictinary.compactify()
    
    corpus = [dictinary.doc2bow(text) for text in texts]
    
    if tf:
        tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)
        corpus = tfidf[corpus]

    lda = gensim.models.LdaModel(corpus, id2word=dictinary, 
                                 passes=passes, num_topics=num_topics,
                                 eta=eta, random_state=42)
    #lda = gensim.models.LdaMulticore(corpus=corpus, num_topics=num_topics,
                                    # workers=workers, passes=passes,
                                   #  eta=eta, iterations=iterations,
                                   #  id2word=dictinary,
                                   #  random_state=42)
    return lda

In [ ]:
lda = my_lda(texts)

Посмотрим на топики.

In [132]:
import pandas as pd


def words(lda):
    d = {}

    for index, line in enumerate(lda.print_topics()):
        d[index] = re.findall('"(.*?)"', line[1])
    
    return pd.DataFrame(d)

In [126]:
words(lda)

,0,1,2,3,4,5,6,7,8,9
0,new,мозг,температура,модуль,страна,массив,страница,услуга,игрок,c
1,класс,боль,машина,заказчик,событие,пакет,браузер,доступ,какой-то,алгоритм
2,свойство,сигнал,режим,архитектура,безопасность,тест,сообщение,трафик,продукт,цикл
3,null,изображение,прибор,управление,доступ,запрос,мобильный,сервис,день,слово
4,id,изменение,скорость,платформа,защита,режим,бот,домен,что-то,ошибка
5,атрибут,происходить,звук,услуга,сотрудник,память,название,база,мир,строка
6,function,учёный,камера,документ,интернет,ядро,контент,канал,понять,блок
7,параметр,область,цена,сервис,правило,узел,поиск,протокол,опыт,переменный
8,поль,её,производитель,изменение,организация,настройка,добавить,передача,деньга,изображение
9,int,пространство,корпус,запрос,специалист,контейнер,пароль,настройка,цель,точка


темы неплохие, но посмотрим что изменится, если изменить параметры

Вообще темы получились неплохие, например, 14 - про обучение, 16,9,5,1 - связанные с программированием, 17 - про убер

50 топиков, 10 passes

In [128]:
lda = my_lda(texts, num_topics=50, passes=10, eta='auto')
words(lda)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,microsoft,услуга,компонент,модуль,температура,google,заказчик,ребёнок,архитектура,свойство,боль,робот,текст,мозг,тест,переменный,товар,узел,питание,function
1,visual_studio,сервис,шаблон,спецификация,скорость,российский,что-то,студент,логика,атрибут,firebase,дом,слово,область,вычислительный,int,покупка,infiniband,контроллер,стиль
2,net,документ,angular,браузер,печать,закон,продукт,младенец,поток,null,изменение,сигнал,перевод,сигнал,виртуальный,дефект,скидка,кластер,температура,рефакторинг
3,azure,дата-центр,react,загрузка,головка,документ,какой-то,участник,presenter,object,сопротивление,управление,название,лицо,производительность,c,магазин,mellanox,корпус,javascript
4,net_core,рынок,div,загрузчик,вода,корпорация,тестирование,родитель,uber,класс,определение,смартфон,английский,изображение,сообщение,дерево,цена,google_cloud,плата,контекст
5,visual,оборудование,страница,квт*ч,датчик,законопроект,понимать,контрольный,состояние,задать,мир,связь,символ,её,задержка,проверка,покупатель,включительно,напряжение,const
6,opencv,предприятие,компонента,изменение,материал,рф,ожидание,отметка,облачный,new,социальный,гаджет,дизайн,свет,операция,ошибка,заказ,неохотно,станция,html
7,xamarin,страна,div_div,поддерживать,диаметр,суд,думать,набор,представление,public,ткань,звук,кнопка,учёный,ядро,оператор,продажа,специально_обученный,ток,логика
8,mac,китай,template,карта,тепло,представитель,правило,предмет,платформа,поль,происходить,база,комментарий,происходить,ос,x,день,56,резистор,документ
9,sql_server,управление,тег,api,принтер,идентификация,понять,группа,бизнес-логика,summary,аудио,телефон,локализация,исследование,алгоритм,выражение,amazon,j_j,светодиод,структура


50 топиков, 5 passes

In [134]:
lda = my_lda(texts, num_topics=50, passes=5, eta='auto')
words(lda)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,продукт,мобильный,клетка,цена,архитектура,доступ,скрипт,услуга,new,контроллер,мозг,модуль,вычислительный,function,температура,товар,google,робот,массив,точка
1,цель,android,страна,диск,логика,безопасность,настройка,сервис,result,питание,сигнал,спецификация,тест,контекст,скорость,скидка,российский,дом,цикл,автомобиль
2,сотрудник,электроника,учёный,4,presenter,защита,контейнер,документ,case,корпус,область,браузер,виртуальный,стиль,головка,заказ,документ,смартфон,c,ребро
3,анализ,аккумулятор,развитие,ноутбук,uber,событие,пароль,дата-центр,java,плата,лицо,загрузка,сообщение,рефакторинг,печать,цена,закон,управление,строка,водитель
4,бизнес,apple,исследование,экран,поток,правило,пакет,рынок,customer,напряжение,изображение,квт*ч,печать,javascript,мощность,amazon,законопроект,звук,алгоритм,3cx
5,исследование,батарея,мир,8,состояние,атака,установить,доступ,3,температура,свет,изменение,задержка,класс,датчик,магазин,рф,гаджет,таблица,рис
6,цифровой,нативный,интернет,производитель,платформа,инцидент,установка,китай,statement,резистор,её,поддерживать,производительность,const,вода,покупатель,корпорация,сигнал,r,луч
7,группа,подборка,ген,смартфон,представление,уязвимость,настроить,предприятие,структура,станция,происходить,карта,операция,html,диаметр,покупка,суд,связь,3,маяк
8,технический,iphone,исследователь,сегодня,бизнес-логика,организация,папка,страна,amount,светодиод,глаз,import,ядро,логика,площадь,акция,представитель,база,n,видимость
9,больший,смартфон,жизнь,дисплей,mvc,репликация,параметр,интернет,result_result,микроконтроллер,пространство,представлять,современный,вычисление,тепло,доставка,идентификация,синтезатор,слово,тест


хорошие темы - 1, 6, 15, 18

100 топиков, 10 passes

In [135]:
lda = my_lda(texts, num_topics=100, passes=10, eta='auto')
words(lda)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,мозг,ansible,пароль,поезд,область,кампания,c,юридически,скидка,порция,массив,домен,rec,проектирование,цифровой,энергия,прибор,библиотека,диспетчер,слайд
1,клетка,opencv,компьютер,земля,обучение,объявление,ошибка,ступень,акция,nmap,цикл,ссылка,swift,блок,открытый,вселенная,двигатель,js,raspberry_pi,тэг
2,ребёнок,маршрутизатор,windows,аппарат,набор,заголовок,плагин,авторский_право,заказ,популяция,репликация,поиск,st,прототип,набор,звезда,метр,ef,синий_зелёный,values
3,учёный,миньон,ключ,орбита,группа,adwords,инструмент,научный,месяц,caché,режим,канал,nintendo,unsigned_char,состояние,учёный,связь,angular,iot,key
4,младенец,employee,домен,км/ч,алгоритм,группа,php,technology,день,шелл,переменный,контент,dx,кнопка_нажать,смысл,её,вода,фреймворка,собрание,ключ
5,область,мастер,доступ,космический,признак,сервис,анализатор,smile,пакет,хищник,r,presenter,swift_3,интерфейс,uber,теория,здание,javascript,hdmi,li
6,тело,salt,скрипт,спутник,машинный_обучение,перенос,resharper,publishing,тариф,question,параметр,скачать,super_mario,этап,рабочий_место,скорость,машина,фреймворк,триггера,signal
7,исследование,вдохновение,windows_10,марс,блокчейн,формат,cmake,research,30,осадок,итерация,музыка,jr,макет,публикация,частица,сигнал,apple,usb,collate
8,пациент,lsa,администратор,миссия,контрольный,ключ,множество,science,промокод,численность,3,релиз,подача_заявка,страница,лицо,галактика,причина,виджет,кружка,args_args
9,организм,роль,настройка,полёт,нейросеть,символ,clion,орбита_спутник,хостинг,arc,пакет,страница,nes,дизайн,знание,существовать,точность,кэш,покой,allowed


хорошие темы - 0, 2, 3, ...

Все темы достаточно хорошие, видимо, с увеличением числа топиков, растет качество их определения

100 топиков, 5 passes

In [136]:
lda = my_lda(texts, num_topics=100, passes=5, eta='auto')
words(lda)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,бот,ия,доход,боль,бар,std::cout,проектирование,рейтинг,бронирование,усилитель,шорткат,библиотека,архитектура,инструмент,страница,пароль,ansible,виртуальный,компонент,диспетчер
1,messages,анализ,налог,камера,скафандр,5,unsigned_char,нервный_волокно,epam,сигнал,сообщение,ef,запрос,музыка,стиль,компьютер,opencv,облачный,div,raspberry_pi
2,сообщение,безопасность,сумма,изменение,болт,6,кнопка_нажать,uint64,юрист,гаджет,android,angular,бизнес-логика,название,документ,подключение,маршрутизатор,среда,react,синий_зелёный
3,php,больший,ип,сопротивление,проигрыватель,3,прототип,third,сергей,звук,действие,js,id,синтезатор,html,доступ,employee,infiniband,div_div,собрание
4,страница,характеристика,расход,определение,пластинка,включительно,блок,page_page,подать_суд,помещение,добавить,apple,view,движок,css,ключ,миньон,ос,шаблон,hdmi
5,twitter,высокий,ооо,ситуация,виниловый,inf,страница,покупка,калькулятор,характеристика,маячок,общественный_мнение,логика,релиз,dom,домен,salt,инфраструктура,тег,0x07
6,id,информационный_безопасность,счёт,видеонаблюдение,шлем,min,хакатон,final_int,василий,фильтр,короткий,vue.js,представление,современный,пиксель,администратор,мастер,виртуальный_машина,компонента,кружка
7,логин,организация,рубль,видео,выступление_спикер,4,этап,idx,два-три,искажение,иконка,фреймворк,поток,серия,ширина,настройка,lsa,производительность,визуальный,триггера
8,форма,искусственный_интеллект,налоговый,происходить,xxi,3_4,макет,string,night,изменение,сутки,кэш,содержимый,библиотека,размер,админ,вдохновение,виртуализация,страница,многое_благодаря
9,public_function,личный,бизнес,сопротивляться,final,int_int,интерфейс,identifier,договор,воздух,телефон,фреймворка,asc,c,main,windows_10,saltstack,физический,angular,pentium


В словах встречаются числа, попробуем их убрать. 

In [148]:
def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {}
    for word, _ in uniq.most_common(top):
        mp = morph.parse(word)[0]
        if 'NUMB' not in mp.tag:
            norm_uniq[word] = mp.normal_form
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

In [149]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
texts = p[texts]

In [197]:
lda = my_lda(texts, num_topics=100, passes=15, eta='auto')
table1 = words(lda)

## для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

Хорошая тема про зрение (лезерную коррекцию зрения)

In [157]:
table1[1]

0    операция
1        глаз
2       лазер
3          тз
4      зрение
5       линза
6        flex
7    роговица
8      хирург
9     пациент
Name: 1, dtype: object

Хорошая тема про учебу

In [158]:
table1[10]

0             студент
1                курс
2    программирование
3             ребёнок
4              лекция
5            обучение
6         программист
7     образовательный
8           компьютер
9              знание
Name: 10, dtype: object

Хорошая тема про здоровье

In [159]:
table1[19]

0        пациент
1           врач
2        лечение
3        болезнь
4    заболевание
5       бактерия
6       медицина
7       организм
8            3cx
9        больной
Name: 19, dtype: object

#### 6) между словарем и обучением модели добавьте tfidf (gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]);
#### 7) повторите пункт 4 на преобразованном корпусе;

In [170]:
lda = my_lda(texts, num_topics=50, passes=10, eta='auto', tf=True)
table = words(lda)

In [171]:
table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,усилитель,вакансия,windows_defender,wp,лвс,боль,std::cout,партнерский,adwords,печать,users,доклад,php,token_token,footer,браузер,кндр,гибкость,yahoo,ipsec
1,поезд,бедный,хромосома,capacity,прокси-сервер,кожа,myclass,forms,pebble,головка,messages,сигнал,else,магнитометр,«вконтакте»,игрок,cs,bitcoin,firewall,vi
2,регулирование,system;using_system.collections.generic;using,dd,критичный_баг,openvpn,квеста,packages,среднестатистический,dart,laravel,clion,процессор,html,8.8.8.8,output_output,сообщение,выпадать_список,main(string_args,listview,sed
3,прибор,middleware,fun,board,shield,гаджет,фасад,«только,орган_государственный,корпус,async/await,подборка,преобразование,intellij,казино,запрос,margin,beyond,беспилотный_автомобиль,tuple
4,балансировщик_нагрузка,утверждаться,hpe,ontap,подключение,realm,регулярно_обновляться,досконально,ёмкость_батарея,изделие,angular,инцидент,нативный,rust,widget,страница,представительство,фреймворок,никак_удаваться,хабов
5,вселенная,сбербанк,reboot,vserver,цифровой_экономика,ecto,занятие_спорт,ansible,pf,apply,cmake,температура,дерево,xe,req,документ,присоединение,cc,gaming,behavior
6,частица,neon,seconds,esxi,vps,наушник,публично_доступный,быстренько,дискета,температура,template,гипотеза,string,traffic,pattern,компонент,игнорирование,pvs-studio,автопилот,syslog
7,галактика,средний_зарплата,деление_клетка,скайп,unsigned_null,подписчик,испытание_двигатель,заводить,fitbit,принтер,flask,слайд,символ,вентилятор,size_t_size,список,ночное,workstation,учётный_запись,бп
8,verizon,вич,se,zend,колонна,рефакторинг,generator,ачх,объектно-ориентированный,филамент,user,питание,const,получас,updated,модуль,новосибирск,обменять,отдаваться,обход_блокировка
9,лампа,подробно_разбирать,коннектор,петербургский,null_default,джон,sim-карта,alexa,s3,пластик,admin,цвет,chrome,снижение_риск,20px,библиотека,input,ребёнок_подросток,faq,постараться_объяснить


#### 8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

Tfidf не улучшил качество работы моделей, а только ухудшил, появились темы, которые ранее не встречались, а аналогов выбранных тем даже нет. Кажется, что слова в темах выбраны рандомно, поэтому не удается выбрать даже одну полноценно хорошую тему. В топ попадает много слов, связанных с программированием(названия функций и тд), возможно, при их удалении картина может измениться, но не очень понятно, как это сделать, если только не убрать все английские слова.

### 9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем


### Разложение матриц в sklearn

In [172]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [173]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [174]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

Разложим её.

In [175]:
model = NMF(n_components=30)

In [176]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [189]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(20):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(model, 10)

Модель выдает хорошие результаты (например, тема 25), попробуем изменить параметры

In [179]:
def my_nmf(stexts, max_features=25000, min_df=0.2, max_df=0.3, lowercase=False):

    vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
    X = vectorizer.fit_transform(stexts)
    
    model = NMF(n_components=30)
    model.fit(X)
    
    return get_nmf_topics(model, 10)

In [191]:
my_nmf(stexts, max_features=5000, min_df=0.0, max_df=0.3)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,программист,скрипт,изображение,windows,смартфон,рынок,атака,спутник,javascript,игрок,класс,доклад,робот,процессор,react,мозг,печать,вселенная,бот,тест
1,сотрудник,настройка,текстура,microsoft,телефон,бизнес,уязвимость,ракета,css,игровой,new,конференция,ребёнок,память,компонент,учёный,принтер,галактика,telegram,тестирование
2,программирование,пакет,алгоритм,linux,android,продукт,безопасность,орбита,js,играть,string,спикер,датчик,ядро,props,пациент,3d,звезда,сообщение,тестовый
3,опыт,папка,точка,обновление,экран,мобильный,злоумышленник,станция,библиотека,персонаж,int,участник,машина,диск,dom,клетка,материал,энергия,чат,тестировать
4,идея,контейнер,карта,ос,аккумулятор,блокчейн,вредоносный,марс,angular,vr,var,рассказать,робототехника,производительность,компонента,исследование,производство,солнце,канал,библиотека
5,книга,установка,координата,visual_studio,мобильный,страна,пароль,космический,vue,враг,public,выступление,движение,intel,render,заболевание,станок,свет,телеграм,ошибка
6,жизнь,директория,пиксель,платформа,дисплей,сервис,защита,земля,api,геймплей,else,встреча,автомобиль,ssd,свойство,ген,деталь,миллиард,api,сценарий
7,день,сборка,размер,драйвер,apple,российский,доступ,наса,веб,мир,amp,тема,дрон,компьютер,jsx,болезнь,печатать,гравитация,мессенджер,tdd
8,курс,docker,слой,уязвимость,корпус,крупный,информационный_безопасность,аппарат,node,виртуальный_реальность,null,мероприятие,способный,накопитель,div,врач,пластик,теория,bot,test
9,знание,amp,анимация,операционный,ноутбук,деньга,угроза,космос,модуль,миссия,void,java,дронов,инструкция,состояние,лечение,изделие,частица,лига,тестировщик


In [192]:
my_nmf(stexts, max_features=10000, min_df=0.3, max_df=1)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,книга,пакет,изображение,windows,смартфон,сотрудник,атака,спутник,мобильный,игрок,класс,доклад,робот,сигнал,пациент,мозг,печать,вселенная,бот,тест
1,программист,настройка,алгоритм,ядро,телефон,бизнес,уязвимость,ракета,продукт,игровой,new,конференция,ребёнок,частота,клетка,учёный,принтер,галактика,сообщение,тестирование
2,программирование,скрипт,текстура,процессор,экран,документ,безопасность,орбита,android,играть,string,спикер,датчик,канал,заболевание,исследование,3d,звезда,telegram,тестовый
3,читать,адрес,точка,память,аккумулятор,деньга,злоумышленник,марс,google,персонаж,int,участник,робототехника,передача,ген,сознание,материал,энергия,канал,тестировать
4,заниматься,ip,карта,microsoft,дисплей,заказчик,вредоносный,космический,реклама,враг,var,выступление,машина,антенна,врач,нейрон,производство,солнце,чат,ошибка
5,жизнь,папка,координата,linux,ноутбук,рынок,пароль,земля,сервис,vr,amp,рассказать,движение,плата,болезнь,область,станок,свет,телеграм,библиотека
6,думать,сертификат,размер,диск,корпус,российский,защита,станция,контент,мир,else,встреча,дрон,ток,лечение,движение,деталь,миллиард,api,сценарий
7,часы,домен,пиксель,ос,apple,продукт,доступ,наса,дизайн,геймплей,public,мероприятие,способный,диапазон,бактерия,активность,печатать,гравитация,мессенджер,производительность
8,идея,установка,слой,intel,батарея,страна,угроза,аппарат,ios,миссия,null,тема,сборка,скорость,организм,исследователь,пластик,теория,сервис,test
9,курс,пароль,анимация,драйвер,цена,крупный,информационный_безопасность,космос,платформа,комната,void,java,автомобиль,связь,учёный,животное,изделие,частица,bot,подход


In [193]:
my_nmf(stexts, max_features=25000, min_df=0.1, max_df=0.6)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,книга,запрос,изображение,пакет,камера,сотрудник,атака,спутник,мобильный,игрок,класс,доклад,робот,процессор,пациент,мозг,печать,вселенная,бот,тест
1,программист,страница,текстура,настройка,смартфон,бизнес,уязвимость,ракета,android,игровой,new,конференция,ребёнок,память,клетка,учёный,принтер,галактика,сообщение,тестирование
2,программирование,таблица,алгоритм,адрес,видео,рынок,безопасность,орбита,google,играть,string,спикер,датчик,диск,заболевание,исследование,3d,звезда,telegram,тестовый
3,идея,база,точка,скрипт,телефон,документ,злоумышленник,станция,продукт,персонаж,var,участник,робототехника,ядро,ген,сознание,материал,энергия,канал,тестировать
4,курс,запись,блок,ip,экран,российский,вредоносный,марс,реклама,враг,public,выступление,машина,производительность,врач,нейрон,производство,солнце,чат,ошибка
5,жизнь,строка,карта,домен,аккумулятор,заказчик,защита,космический,ios,vr,null,рассказать,движение,intel,болезнь,область,станок,свет,телеграм,библиотека
6,думать,бд,координата,сертификат,корпус,деньга,пароль,земля,сервис,геймплей,id,встреча,дрон,ssd,лечение,движение,деталь,миллиард,мессенджер,сценарий
7,знание,индекс,анимация,установка,дисплей,страна,доступ,наса,контент,мир,class,мероприятие,автомобиль,вм,бактерия,активность,печатать,гравитация,api,test
8,понять,документ,размер,папка,регистратор,продукт,угроза,аппарат,рынок,миссия,private,тема,способный,накопитель,учёный,животное,пластик,теория,сервис,баг
9,опыт,ответ,пиксель,пароль,датчик,крупный,ботнет,космос,дизайн,комната,name,java,сборка,запись,организм,исследователь,изделие,частица,bot,tdd


In [194]:
my_nmf(stexts, max_features=25000, min_df=0.3, max_df=0.1)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,сотрудник,запрос,изображение,диск,смартфон,мобильный,атака,спутник,react,игрок,класс,доклад,робот,процессор,мозг,ия,печать,вселенная,бот,тест
1,продукт,скрипт,текстура,вм,телефон,android,уязвимость,ракета,javascript,игровой,new,конференция,ребёнок,память,учёный,нейросеть,принтер,галактика,сообщение,тестирование
2,бизнес,страница,карта,виртуальный_машина,аккумулятор,продукт,безопасность,орбита,компонент,играть,string,спикер,датчик,ядро,пациент,искусственный_интеллект,3d,звезда,telegram,тестовый
3,деньга,пакет,точка,виртуальный,экран,google,злоумышленник,станция,css,персонаж,var,участник,машина,intel,клетка,обучение,материал,энергия,канал,тестировать
4,день,адрес,алгоритм,запись,дисплей,реклама,пароль,марс,js,враг,public,выступление,робототехника,производительность,исследование,deepmind,производство,солнце,чат,библиотека
5,опыт,настройка,координата,бэкап,корпус,ios,вредоносный,космический,angular,vr,null,рассказать,движение,модуль,заболевание,google,станок,свет,телеграм,ошибка
6,заказчик,строка,пиксель,резервный_копирование,ноутбук,дизайн,защита,земля,библиотека,мир,id,встреча,дрон,регистр,ген,машинный_обучение,деталь,миллиард,мессенджер,test
7,менеджер,список,анимация,база,батарея,контент,доступ,наса,браузер,геймплей,class,мероприятие,автомобиль,инструкция,болезнь,нейронный,печатать,гравитация,api,сценарий
8,жизнь,сервис,слой,восстановление,apple,платформа,шифрование,аппарат,dom,миссия,private,тема,сборка,архитектура,врач,агент,пластик,теория,сервис,tdd
9,понять,таблица,размер,хранилище,гаджет,сервис,угроза,космос,html,комната,name,слайд,способный,поток,лечение,машина,изделие,частица,bot,производительность


In [195]:
my_nmf(stexts, max_features=25000, min_df=0.0, max_df=0.3)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,программист,скрипт,изображение,мобильный,камера,бизнес,сообщение,спутник,react,игрок,класс,доклад,ия,процессор,пациент,мозг,печать,вселенная,бот,атака
1,книга,пакет,текстура,android,смартфон,сотрудник,домен,ракета,javascript,игровой,new,конференция,нейросеть,память,клетка,учёный,принтер,галактика,telegram,уязвимость
2,программирование,настройка,алгоритм,google,видео,рынок,адрес,орбита,компонент,играть,string,спикер,искусственный_интеллект,ядро,заболевание,исследование,3d,звезда,сообщение,безопасность
3,идея,папка,точка,продукт,телефон,блокчейн,сервис,станция,css,персонаж,int,участник,обучение,intel,ген,сознание,материал,энергия,чат,злоумышленник
4,курс,php,карта,ios,экран,страна,письмо,марс,js,враг,var,выступление,deepmind,производительность,врач,нейрон,производство,солнце,канал,пароль
5,опыт,установка,координата,реклама,корпус,услуга,сертификат,космический,angular,vr,public,рассказать,google,регистр,болезнь,область,станок,свет,телеграм,вредоносный
6,жизнь,модуль,пиксель,дизайн,аккумулятор,российский,ip,земля,браузер,геймплей,amp,встреча,машинный_обучение,инструкция,лечение,движение,деталь,миллиард,api,защита
7,день,директория,размер,платформа,дисплей,деньга,трафик,наса,библиотека,мир,else,мероприятие,нейронный,компьютер,бактерия,активность,печатать,гравитация,мессенджер,доступ
8,знание,сборка,слой,рынок,регистратор,крупный,провайдер,аппарат,dom,миссия,null,тема,автомобиль,модуль,организм,исследователь,пластик,теория,bot,ботнет
9,думать,контейнер,анимация,контент,датчик,документ,канал,космос,html,комната,void,java,компьютер,чип,учёный,животное,изделие,частица,лига,угроза


Кажется, что вне зависимости от значения пар-ор модели строят отличные темы, лично мне больше всего понравились при max_features=5000, min_df=0.0, max_df=0.3

### 10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

In [199]:
t1 = my_nmf(stexts, max_features=5000, min_df=0.0, max_df=0.3)

In [201]:
t1 = t1[['Topic # 08', 'Topic # 14', 'Topic # 15']]

In [198]:
t2 = table1[[1, 10, 19]]

In [203]:
pd.concat([t1, t2], axis=1)

,Topic # 08,Topic # 14,Topic # 15,1,10,19
0,спутник,товар,пациент,операция,студент,пациент
1,ракета,скидка,клетка,глаз,курс,врач
2,орбита,цена,заболевание,лазер,программирование,лечение
3,станция,магазин,ген,тз,ребёнок,болезнь
4,марс,покупатель,врач,зрение,лекция,заболевание
5,космический,покупка,болезнь,линза,обучение,бактерия
6,земля,распродажа,лечение,flex,программист,медицина
7,наса,продажа,бактерия,роговица,образовательный,организм
8,аппарат,продавец,организм,хирург,компьютер,3cx
9,космос,акция,днк,пациент,знание,больной


1) Не смотря на то, что LDA выбрал досточно много неплохо характеризующих слов для тем, в списке все равно попал шум-мусор (3cx, тз, flex)
В NMF же такого не наблюдается. 

2) В NMF слова четко специфицированы под тему, даже сложно выбрать 3 лучших, тогда как, например, в 2 теме LDA, слова вроде как по теме, а вроде как бы и не очень.

3) Всякие термины связанные с программированием NMF объединил в одну значимую тему, а LDA вставлял подобные слова в соверщенно разные темы.